In [32]:
from os import listdir
import os
import math
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import tensorflow as tf

In [33]:
os.getcwd()

'/Users/dominikpeter/Homework/mining/Tensorflow'

In [34]:
x = [x for x in os.walk('/Users/dominikpeter/Homework/mining/Tensorflow/HMP_Dataset')]

In [35]:
subfolder = x[0][1]
subfolder[0:5]

['Brush_teeth',
 'Climb_stairs',
 'Climb_stairs_MODEL',
 'Comb_hair',
 'Descend_stairs']

In [36]:
folders = []
[folders.append('/Users/dominikpeter/Homework/mining/Tensorflow/HMP_Dataset' + '/' + word) for word in subfolder]
folders[0:5]

['/Users/dominikpeter/Homework/mining/Tensorflow/HMP_Dataset/Brush_teeth',
 '/Users/dominikpeter/Homework/mining/Tensorflow/HMP_Dataset/Climb_stairs',
 '/Users/dominikpeter/Homework/mining/Tensorflow/HMP_Dataset/Climb_stairs_MODEL',
 '/Users/dominikpeter/Homework/mining/Tensorflow/HMP_Dataset/Comb_hair',
 '/Users/dominikpeter/Homework/mining/Tensorflow/HMP_Dataset/Descend_stairs']

In [37]:
files = []
for folder in folders:
    [files.append(folder + '/' + f) for f in listdir(folder) if f.endswith(".txt")]
files[0:5]

['/Users/dominikpeter/Homework/mining/Tensorflow/HMP_Dataset/Brush_teeth/Accelerometer-2011-04-11-13-28-18-brush_teeth-f1.txt',
 '/Users/dominikpeter/Homework/mining/Tensorflow/HMP_Dataset/Brush_teeth/Accelerometer-2011-04-11-13-29-54-brush_teeth-f1.txt',
 '/Users/dominikpeter/Homework/mining/Tensorflow/HMP_Dataset/Brush_teeth/Accelerometer-2011-05-30-08-35-11-brush_teeth-f1.txt',
 '/Users/dominikpeter/Homework/mining/Tensorflow/HMP_Dataset/Brush_teeth/Accelerometer-2011-05-30-09-36-50-brush_teeth-f1.txt',
 '/Users/dominikpeter/Homework/mining/Tensorflow/HMP_Dataset/Brush_teeth/Accelerometer-2011-05-30-10-34-16-brush_teeth-m1.txt']

In [38]:
df = []
for file in files:
    new_df = pd.read_csv(file, sep = ' ', header=None, names = ['x', 'y', 'z'])
    new_df['label'] = re.search('-\w+-', file[::-1]).group(0)[::-1].replace('-', '')
    df.append(new_df)
df = pd.concat(df, axis=0)
df.head()

,x,y,z,label
0,22,49,35,brush_teeth
1,22,49,35,brush_teeth
2,22,52,35,brush_teeth
3,22,52,35,brush_teeth
4,21,52,34,brush_teeth


In [39]:
df.shape

(479289, 4)

In [40]:
df = df.join(df.groupby('label')[['x', 'y','z']].mean(), on='label', rsuffix='.mean')
df.head()

,x,y,z,label,x.mean,y.mean,z.mean
0,22,49,35,brush_teeth,27.654397,49.281505,34.500654
1,22,49,35,brush_teeth,27.654397,49.281505,34.500654
2,22,52,35,brush_teeth,27.654397,49.281505,34.500654
3,22,52,35,brush_teeth,27.654397,49.281505,34.500654
4,21,52,34,brush_teeth,27.654397,49.281505,34.500654


In [41]:
df['diff.x'] = abs(df['x'] - df['x.mean'])
df['diff.y'] = abs(df['y'] - df['y.mean'])
df['diff.z'] = abs(df['z'] - df['z.mean'])
df['max.y'] = max(df['y'])

In [42]:
df.head()

,x,y,z,label,x.mean,y.mean,z.mean,diff.x,diff.y,diff.z,max.y
0,22,49,35,brush_teeth,27.654397,49.281505,34.500654,5.654397,0.281505,0.499346,63
1,22,49,35,brush_teeth,27.654397,49.281505,34.500654,5.654397,0.281505,0.499346,63
2,22,52,35,brush_teeth,27.654397,49.281505,34.500654,5.654397,2.718495,0.499346,63
3,22,52,35,brush_teeth,27.654397,49.281505,34.500654,5.654397,2.718495,0.499346,63
4,21,52,34,brush_teeth,27.654397,49.281505,34.500654,6.654397,2.718495,0.500654,63


In [43]:
le = preprocessing.LabelEncoder()
le.fit(df['label'])
le.classes_

array(['brush_teeth', 'climb_stairs', 'comb_hair', 'descend_stairs',
       'drink_glass', 'eat_meat', 'eat_soup', 'getup_bed', 'liedown_bed',
       'pour_water', 'sitdown_chair', 'standup_chair', 'use_telephone',
       'walk'], dtype=object)

In [44]:
df['class'] = le.transform(df['label'])

In [45]:
y = pd.get_dummies(df['class'])
y.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [46]:
X = df.iloc[:, [0,1,2,4,5,6,7,8,9]]
#X = df.iloc[:, [0,1,2]]

In [47]:
X = X.as_matrix()
y = y.as_matrix()

In [48]:
normalizer = preprocessing.Normalizer().fit(X)
X = normalizer.transform(X)
#scaler = preprocessing.StandardScaler().fit(X)
#X = scaler.transform(X)

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [50]:
print(X.shape, y.shape)

(479289, 9) (479289, 14)


In [51]:
x = tf.placeholder(tf.float32, [None, X.shape[1]])

In [52]:
y.shape[1]
X.shape[1]

9

In [53]:
W = tf.Variable(tf.zeros([X.shape[1], y.shape[1]]))
b = tf.Variable(tf.zeros([y.shape[1]]))

In [54]:
y = tf.nn.softmax(tf.matmul(x, W) + b)
y_ = tf.placeholder(tf.float32, [None, y.shape[1]])

In [55]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

In [56]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [57]:
sess = tf.InteractiveSession()

In [58]:
tf.global_variables_initializer().run()

In [59]:
i = 0
while i < 100:
    for start, end in zip(range(0, len(X_train), 150), range(150, len(X_train)+1, 150)):
            sess.run(train_step, feed_dict={x: X_train[start:end], y_: y_train[start:end]})
    i += 1

In [63]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [64]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [65]:
print(sess.run(accuracy, feed_dict={x: X_test, y_: y_test}))

0.820088
